<a href="https://colab.research.google.com/github/Ramanarayanan/Agentic-AI/blob/main/Travel_planner_Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install crewai==0.28.8 crewai-tools==0.1.6 serpapi google-generativeai IPython groq langchain-groq crewai-tools



In [83]:
import os
from google.colab import userdata

# Load API Keys from Colab Secrets
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key_valu')

# Check if keys are loaded
if not os.environ["SERPER_API_KEY"] or not os.environ["GROQ_API_KEY"]:
    print("One or more API keys not found. Please add them to Colab Secrets.")
else:
    print("API keys loaded successfully.")



API keys loaded successfully.


In [84]:
!pip install google-search-results



In [85]:
from serpapi import GoogleSearch

In [86]:
import json
from datetime import datetime

def format_duration(minutes):
    hours = minutes // 60
    mins = minutes % 60
    return f"{hours}h {mins}m"

def format_flight_data(data):
    formatted_output = []
    for index, itinerary in enumerate(data, start=1):
        flights = itinerary['flights']
        total_duration = format_duration(itinerary['total_duration'])
        carbon = itinerary['carbon_emissions']
        price = itinerary['price']
        itinerary_type = itinerary['type']
        logo_url = itinerary['airline_logo']

        formatted_output.append(f"--- Itinerary {index} ({itinerary_type}) ---")
        for flight in flights:
            dep = flight['departure_airport']
            arr = flight['arrival_airport']
            duration = format_duration(flight['duration'])
            airline = flight['airline']
            airplane = flight['airplane']
            travel_class = flight['travel_class']
            flight_number = flight['flight_number']
            legroom = flight.get('legroom', 'N/A')
            extensions = ", ".join(flight.get('extensions', []))
            overnight = flight.get('overnight', False)

            formatted_output.append(
                f"Flight: {flight_number} ({airline})\n"
                f"  From: {dep['name']} ({dep['id']}) at {dep['time']}\n"
                f"  To:   {arr['name']} ({arr['id']}) at {arr['time']}\n"
                f"  Duration: {duration} | Class: {travel_class} | Aircraft: {airplane}\n"
                f"  Legroom: {legroom}\n"
                f"  Extras: {extensions}"
            )
            if overnight:
                formatted_output.append("  ✦ Overnight flight")

        emission_diff = carbon['difference_percent']
        formatted_output.append(
            f"Carbon Emissions: {carbon['this_flight'] // 1000} kg "
            f"({emission_diff}% vs typical: {carbon['typical_for_this_route'] // 1000} kg)"
        )
        formatted_output.append(f"Price: ${price}")
        formatted_output.append(f"Airline Logo: {logo_url}\n")

    return "\n".join(formatted_output)

In [101]:
def format_flight_data_Content(data):
    formatted_output = []
    for index, itinerary in enumerate(data, start=1):
        flights = itinerary['flights']
        total_duration = format_duration(itinerary['total_duration'])
        carbon = itinerary['carbon_emissions']
        price = itinerary['price']
        itinerary_type = itinerary['type']
        logo_url = itinerary['airline_logo']

        formatted_output.append(f"--- Itinerary {index} ({itinerary_type}) ---")
        for flight in flights:
            dep = flight['departure_airport']
            arr = flight['arrival_airport']
            duration = format_duration(flight['duration'])
            airline = flight['airline']
            airplane = flight['airplane']
            travel_class = flight['travel_class']
            flight_number = flight['flight_number']
            legroom = flight.get('legroom', 'N/A')
            extensions = ", ".join(flight.get('extensions', []))
            overnight = flight.get('overnight', False)

            formatted_output.append(
                f"Flight: {flight_number} ({airline})\n"
                f"  From: {dep['name']} ({dep['id']}) at {dep['time']}\n"
                f"  To:   {arr['name']} ({arr['id']}) at {arr['time']}\n"
                f"  Duration: {duration} | Class: {travel_class} | Aircraft: {airplane}\n"
                f"  Legroom: {legroom}\n"
                f"  Extras: {extensions}"
            )
            if overnight:
                formatted_output.append("  ✦ Overnight flight")

        emission_diff = carbon['difference_percent']
        formatted_output.append(
            f"Carbon Emissions: {carbon['this_flight'] // 1000} kg "
            f"({emission_diff}% vs typical: {carbon['typical_for_this_route'] // 1000} kg)"
        )
        formatted_output.append(f"Price: ${price}")
        formatted_output.append(f"Airline Logo: {logo_url}\n")

    return formatted_output

In [87]:
def format_property_info(prop):
    lines = []
    lines.append(f"🏡 Name: {prop.get('name')}")
    lines.append(f"🏷️ Type: {prop.get('type')}")
    lines.append(f"⭐ Rating: {prop.get('overall_rating')} stars ({prop.get('reviews')} reviews)")
    lines.append(f"📍 Location Rating: {prop.get('location_rating')}")
    lines.append(f"🕒 Check-in: {prop.get('check_in_time')} | Check-out: {prop.get('check_out_time')}")

    # Prices - Iterate and safely access nested keys
    prices = prop.get('prices', [])
    if prices:
        # Try to get the first price entry's rate per night and total rate
        first_price_entry = prices[0]
        rate_per_night_info = first_price_entry.get('rate_per_night', {})
        rate_per_night_lowest = rate_per_night_info.get('lowest', 'N/A')
        total_rate_info = first_price_entry.get('total_rate', {})
        total_rate_lowest = total_rate_info.get('lowest', 'N/A')
        lines.append(f"💵 Rate per Night: {rate_per_night_lowest} | Total: {total_rate_lowest}")

        # Add additional prices if available
        for price in prices[1:]: # Start from the second entry if exists
            cancellation_info = (
                f" | Free Cancellation until {price.get('free_cancellation_until_date', 'N/A')} {price.get('free_cancellation_until_time', 'N/A')}"
                if price.get('free_cancellation') else ""
            )
            # Safely access nested keys for additional prices
            additional_rate_per_night_info = price.get('rate_per_night', {})
            additional_rate_per_night = additional_rate_per_night_info.get('lowest', 'N/A')
            lines.append(
                f"💲 Price from {price.get('source', 'N/A')}: {additional_rate_per_night} per night for {price.get('num_guests', 'N/A')} guests{cancellation_info}"
            )

    else:
        lines.append("💵 Rate per Night: N/A | Total: N/A")


    # Safely access gps_coordinates
    gps_coords = prop.get('gps_coordinates', {})
    latitude = gps_coords.get('latitude', 'N/A')
    longitude = gps_coords.get('longitude', 'N/A')
    lines.append(f"🗺️ Coordinates: ({latitude}, {longitude})")


    lines.append(f"📋 Essential Info: {', '.join(prop.get('essential_info', []))}")
    lines.append(f"✅ Amenities: {', '.join(prop.get('amenities', []))}")
    lines.append(f"❌ Excluded Amenities: {', '.join(prop.get('excluded_amenities', []))}")


    # Nearby Places
    if prop.get('nearby_places'):
        nearby = []
        for place in prop['nearby_places']:
            transport = ', '.join([f"{t.get('type', 'N/A')} ({t.get('duration', 'N/A')})" for t in place.get('transportations', [])])
            nearby.append(f"{place.get('name', 'N/A')} via {transport}")
        lines.append("📌 Nearby Places: " + "; ".join(nearby))

    # Link
    lines.append(f"🔗 Details: {prop.get('serpapi_property_details_link')}")

    return "\n".join(lines)

In [112]:
import logging
try:
    from serpapi.google_search import GoogleSearch
except ImportError:
    logging.error("The serpapi library is not installed correctly. Please ensure you have installed 'serpapi' and 'crewai-tools' which includes the serpapi tool.")
    GoogleSearch = None # Define GoogleSearch as None to avoid further errors

from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq
from datetime import datetime
from IPython.display import display, Markdown

# Initialize Logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Initialize the LLM (Groq)
llm = ChatGroq(
    temperature=0.7,
    model_name="gemma2-9b-it", # Or "mixtral-8x7b-32768"
    api_key=os.environ["GROQ_API_KEY"]
)


def search_flights(origin, destination, outbound_date, return_date):
    """Fetch real-time flight details from Google Flights using SerpAPI."""
    if GoogleSearch is None:
        logger.error("GoogleSearch is not available due to import error.")
        return []
    logger.info(f"Searching flights: {origin} to {destination}")
    params = {
        "api_key": os.environ["SERPER_API_KEY"],
        "engine": "google_flights",
        "hl": "en",
        "gl": "us",
        "departure_id": origin.strip().upper(),
        "arrival_id": destination.strip().upper(),
        "outbound_date": outbound_date,
        "return_date": return_date,
        "currency": "USD"
    }
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        return results.get("best_flights", [])
    except Exception as e:
        logger.error(f"Flight search error: {e}")
        return []

def search_hotels(location, check_in_date, check_out_date):
    """Fetch hotel information from SerpAPI."""
    if GoogleSearch is None:
        logger.error("GoogleSearch is not available due to import error.")
        return []
    logger.info(f"Searching hotels for: {location}")
    params = {
        "api_key": os.environ["SERPER_API_KEY"],
        "engine": "google_hotels",
        "q": f"hotels in {location}",
        "hl": "en",
        "gl": "us",
        "check_in_date": check_in_date,
        "check_out_date": check_out_date,
        "currency": "INR"
    }
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        return results.get("properties", [])
    except Exception as e:
        logger.error(f"Hotel search error: {e}")
        return []

def format_travel_data(data_type, data_list):
    """Format flight or hotel data into a readable string for the AI."""
    if not data_list:
        return "No data available."
    formatted_string = ""
    if data_type == "flights":
        return format_flight_data(data_list)
        # for i, flight in enumerate(data_list[:5]): # Limit to top 5 for brevity
        #     formatted_string += f"Flight {i+1}: Airline: {flight.get('flights', [{}])[0].get('airline')}, Price: {flight.get('price')}, Stops: {flight.get('flights', [{}])[0].get('stops', 'N/A')}, Duration: {flight.get('total_duration')}\\n"
        #     print(formatted_string)
    elif data_type == "hotels":
        print("--- Raw Hotel Data ---") # Add print statement here
        # print(json.dumps(data_list, indent=2)) # Pretty print the raw data
        print("--- End Raw Hotel Data ---") # Add print statement here

        # Check if data_list is not empty before accessing the first element
        if data_list:
            formatted_string = format_property_info(data_list[0])
        else:
            formatted_string = "No hotel data available to format."

    return formatted_string

def get_ai_recommendation(data_type, formatted_data):
    """Get AI recommendation for flights or hotels using a CrewAI agent."""
    if data_type == "flights":
        role = "AI Flight Analyst"
        goal = "Analyze flight options and recommend the best one considering price, duration, stops, and overall convenience."
        backstory = f"AI expert that provides in-depth analysis comparing flight options based on multiple factors."
        description = """
        Recommend the best flight from the available options, based on the details provided below:

        **Reasoning for Recommendation:**
        - **Price:** Provide a detailed explanation about why this flight offers the best value compared to others.
        - **Duration:** Explain why this flight has the best duration in comparison to others.
        - **Stops:** Discuss why this flight has minimal or optimal stops.
        - **Travel Class:** Describe why this flight provides the best comfort and amenities.

        Use the provided flight data as the basis for your recommendation. Be sure to justify your choice using clear reasoning for each attribute. Do not repeat the flight details in your response.
        """
        # role = "AI Flight Analyst"
        # goal = "Analyze flight options and recommend the best one based on price, duration, and stops."
        # backstory = "You are an expert flight analyst who provides clear, data-driven recommendations."
    else:
        # role = "AI Hotel Analyst"
        # goal = "Analyze hotel options and recommend the best one based on price, rating, and location."
        # backstory = "You are a savvy travel expert who helps users find the perfect hotel."
        role = "AI Hotel Analyst"
        goal = "Analyze hotel options and recommend the best one considering price, rating, location, and amenities."
        backstory = f"AI expert that provides in-depth analysis comparing hotel options based on multiple factors."
        description = """
        Based on the following analysis, generate a detailed recommendation for the best hotel. Your response should include clear reasoning based on price, rating, location, and amenities.

        **AI Hotel Recommendation**
        We recommend the best hotel based on the following analysis:

        **Reasoning for Recommendation**:
        - **Price:** The recommended hotel is the best option for the price compared to others, offering the best value for the amenities and services provided.
        - **Rating:** With a higher rating compared to the alternatives, it ensures a better overall guest experience. Explain why this makes it the best choice.
        - **Location:** The hotel is in a prime location, close to important attractions, making it convenient for travelers.
        - **Amenities:** The hotel offers amenities like Wi-Fi, pool, fitness center, free breakfast, etc. Discuss how these amenities enhance the experience, making it suitable for different types of travelers.

        **Reasoning Requirements**:
        - Ensure that each section clearly explains why this hotel is the best option based on the factors of price, rating, location, and amenities.
        - Compare it against the other options and explain why this one stands out.
        - Provide concise, well-structured reasoning to make the recommendation clear to the traveler.
        - Your recommendation should help a traveler make an informed decision based on multiple factors, not just one.
        """

    analyst_agent = Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        llm=llm,
        verbose=False
    )

    analyze_task = Task(
        description=f"Analyze the following {data_type} data and provide a single best recommendation from {formatted_data}. Explain your choice clearly, focusing on the most important factors.\n\nData:\n{formatted_data}",
        agent=analyst_agent,
        expected_output=f"A single, concise recommendation for the best {data_type[:-1]} with a brief justification."
    )

    crew = Crew(
        agents=[analyst_agent],
        tasks=[analyze_task],
        process=Process.sequential,
        verbose=False
    )

    result = crew.kickoff()
    return result

def generate_itinerary(destination, flights_text, hotels_text, check_in_date, check_out_date,flight_Summary):
    """Generate a travel itinerary using a CrewAI agent."""
    days = (datetime.strptime(check_out_date, "%Y-%m-%d") - datetime.strptime(check_in_date, "%Y-%m-%d")).days

    itinerary_agent = Agent(
        role="AI Travel Planner",
        goal=f"Create a detailed, day-by-day itinerary for a {days}-day trip to {destination}.and show the all flight summary from {flight_Summary}",
        backstory="You are a world-class travel planner who creates exciting and practical itineraries.",
        llm=llm,
        verbose=False
    )

    itinerary_task = Task(
        description=f"""
            Based on the following details, create a {days}-day itinerary for the user:
            Incorporate the following details:
            **All Flight Details Summary**
            {flight_Summary}
            **Flight Details**:
            {flights_text}

            **Hotel Details**:
            {hotels_text}

            **Destination**: {destination}

            **Travel Dates**: {check_in_date} to {check_out_date} ({days} days)
         The itinerary should include:
            - All Flight summary
            - Flight arrival and departure information
            - Hotel check-in and check-out details
            - Day-by-day breakdown of activities
            - Must-visit attractions and estimated visit times
            - Recommendations for breakfast, lunch, and dinner each day.
            - Tips for local transportation

            **Format Requirements**:
            - Use markdown formatting with clear headings (# for main headings, ## for days, ### for sections)
            - Include emojis for different types of activities ( for landmarks, 🍽️ for restaurants, etc.)
            - Use bullet points for listing activities
            - Include estimated timings for each activity
            - Format the itinerary to be visually appealing and easy to read
            """,
        agent=itinerary_agent,
        expected_output="A well-structured, visually appealing itinerary in markdown format, including flight, hotel, and day-wise breakdown with emojis, headers, and bullet points."
        # description=f"""Create a {days}-day itinerary for a trip to {destination} from {check_in_date} to {check_out_date}.
        # Incorporate the following details:
        # - Flight Info: {flights_text}
        # - Hotel Info: {hotels_text}

        # Your itinerary should include:
        # - Must-visit attractions with estimated visit times.
        # - Recommendations for breakfast, lunch, and dinner each day.
        # - Tips for local transportation.
        # - Use Markdown for formatting with clear headings for each day.""",
        # agent=itinerary_agent,
        # expected_output="A well-structured itinerary in Markdown format."
    )

    crew = Crew(
        agents=[itinerary_agent],
        tasks=[itinerary_task],
        process=Process.sequential,
        verbose=False
    )
    result = crew.kickoff()
    return result

In [89]:
from datetime import date

# Get today's date
today = date.today()

def get_ExtractContent(content):
    """
    Uses an AI agent to find trim details from the contact.
    """
    logger.info(f"Finding airport code for: {content}")

    # Create an agent specifically for finding airport codes
    code_finder_agent = Agent(
        role='Trip planner data analyst',
        goal=f'Extract the the origin city name,trip orgin destination name,Departure name,Return date for trip planning from {content}. ',
        backstory='An Trip planner analyst and data anlyst, skilled at extracting trip planning details for trip planning.',
        # tools=[search_tool], # This tool is not defined, removing it
        llm=llm,
        verbose=False
    )
    trip_analyst_agent = Agent(
    role='Trip Details Extractor',
    goal=f'Accurately extract the origin city, destination city, departure date, and return date from {content}.',
    backstory=(
        "You are an expert data analyst working for a travel company. "
        "Your primary function is to read raw text from user requests and "
        "convert it into a structured JSON format that the booking system can understand."
    ),
    llm=llm, # In a real scenario, you assign your configured LLM here.
    verbose=False,
    allow_delegation=False,
)
    code_finder_task = Task(
        description=f'Extract the the origin city name,trip orgin destination name,Departure name,Return date from for trip planning .',
        agent=code_finder_agent,
        expected_output='A json with keys TRIP_ORIGIN, TRIP_DESTINATION, DEPARTURE_DATE, and RETURN_DATE. Dates must be in YYYY-MM-DD format.'
    )
    extraction_task = Task(
    description=(
        "Analyze the following trip planning request and extract the key details.\n"
        "--- REQUEST ---\n"
        "{content}\n"
        "--- END REQUEST ---"
    ),
    expected_output=(
       "A json with keys originCity, destinationCity, departureDate, and returnDate. Dates must be in YYYY-MM-DD format"
    ),
    agent=trip_analyst_agent,
)

    # Create and run the crew
    crew = Crew(
        agents=[trip_analyst_agent],
        tasks=[extraction_task],
        process=Process.sequential

    )

    result = crew.kickoff()
    logger.info(f"Found airport code for {content}: {result}")
    print(type(result.strip()))
    return result

In [90]:
from datetime import date

# Get today's date
today = date.today()

def adjust_date_year(date_str: str) -> str:
    """
    Adjusts the year for a date string if it's missing or in the past.

    If the date's month and day are less than or equal to today's date,
    it sets the year to the upcoming year. If the date includes a year that is before
    the current date, it sets the year to the current or upcoming year as needed.

    Args:
        date_str: The date string from the agent (e.g., "09-27", "2024-10-15", or "10-15").

    Returns:
        A date string in "YYYY-MM-DD" format.
    """
    try:
        # Check if the year is already present
        if len(date_str.split('-')) == 3:
            year, month, day = map(int, date_str.split('-'))
            extracted_date = date(year, month, day)
            current_year = today.year

            # If the extracted date is in the past, move it to the current or next year
            if extracted_date < today:
                # Try setting to the current year first
                date_this_year = date(current_year, month, day)
                if date_this_year < today:
                    # If date in current year is still in the past, use next year
                    new_year = current_year + 1
                else:
                    # Use current year
                    new_year = current_year
                return f"{new_year}-{month:02d}-{day:02d}"
            else:
                return date_str # Date is in the future or is today, no change needed

        # If year is missing (format is MM-DD)
        month_str, day_str = date_str.split('-')
        month, day = int(month_str), int(day_str)

        current_year = today.year
        # Construct date object for the current year
        date_this_year = date(current_year, month, day)

        # If the date in the current year is less than or equal to today, assume the upcoming year
        if date_this_year <= today:
            new_year = current_year + 1
        else:
            new_year = current_year

        return f"{new_year}-{month:02d}-{day:02d}"

    except (ValueError, IndexError):
        # Return original string if it's not in a recognizable MM-DD or YYYY-MM-DD format
        return date_str

In [91]:
def processJsonContentForTripPlan(extracted_Data):
    cleaned_output = extracted_Data.strip()
    if cleaned_output.startswith('```json'):
        cleaned_output = cleaned_output[7:]
    if cleaned_output.endswith('```'):
        cleaned_output = cleaned_output[:-3]
    cleaned_output = cleaned_output.strip().replace('\n', '') # Remove any remaining leading/trailing whitespace and newlines



    try:
        # Parse the JSON string into a Python dictionary
        trip_data = json.loads(cleaned_output)
        print("\nSuccessfully parsed JSON.")


        # 2. Adjust the departure and return dates
        if 'departureDate' in trip_data:
          trip_data['departureDate'] = adjust_date_year(trip_data['departureDate'])

        if 'returnDate' in trip_data:
          trip_data['returnDate'] = adjust_date_year(trip_data['returnDate'])

        # 3. Return the cleaned JSON as a string
        final_result = json.dumps(trip_data)

    except json.JSONDecodeError as e:
        print(f"\nJSONDecodeError: Could not parse the cleaned output as JSON. Error: {e}")
        print(f"String that caused the error: '{cleaned_output}'")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
    return final_result

In [92]:
def get_airport_code(city_name):
    """
    Uses an AI agent to find the IATA airport code for a given city name.
    """
    logger.info(f"Finding airport code for: {city_name}")

    # Create an agent specifically for finding airport codes
    code_finder_agent = Agent(
        role='Airport Code Researcher',
        goal=f'Find the primary IATA airport code for {city_name}.',
        backstory='An expert in aviation and travel data, skilled at finding precise airport codes from web searches.',
        llm=llm,
        verbose=False
    )
    code_finder_task = Task(
        description=f'Search for the main international airport IATA code for the city: "{city_name}". Return only the three-letter IATA code and nothing else.',
        agent=code_finder_agent,
        expected_output='A single, three-letter IATA airport code (e.g., "SFO").'
    )

    # Create and run the crew
    crew = Crew(
        agents=[code_finder_agent],
        tasks=[code_finder_task],
        process=Process.sequential,
        verbose=False
    )

    result = crew.kickoff()
    # logger.info(f"Found airport code for {city_name}: {result}")
    return result.strip()

In [103]:
def flight_Summary(content): # Added the missing colon here

    # 2. Create summarizer agent
    flight_summarizer = Agent(
        role="Flight Details Summarizer",
        goal=f"Read detailed flight options and produce a concise summary for a traveler from {content}.",
        backstory=(
            "You are an expert travel assistant who quickly distills multiple flight "
            "options into clear summaries highlighting the best choices."
        ),
        verbose=True,
        memory=True,
        allow_delegation=False,
        # Choose a Groq-supported model (e.g., LLaMA or Mixtral)
        llm=llm
    )

    # 3. Define summarization task
    summarize_flights_task = Task(
        description=(
            "Summarize the following available flights for the trip:\n"
            "--- FLIGHTS ---\n"
            "{content}\n"
            "--- END FLIGHTS ---\n"
            "Provide a concise summary with the cheapest and earliest options highlighted."
        ),
        expected_output="A 3–4 sentence summary of the key flight details, highlighting cheapest and earliest flights and show all flight details in rich look.",
        agent=flight_summarizer,
    )

    # 4. Create the crew
    crew = Crew(
        agents=[flight_summarizer],
        tasks=[summarize_flights_task],
        process=Process.sequential
    )

    # 5. Run it
    result = crew.kickoff()
    print("\n=== Flight Summary ===\n")
    print(result)
    return result

In [ ]:
text ="Plan a trip from chennai to USA from jan 20 to feb 20 next year"
OrchesterAgentsAndTask(text)

In [109]:
def processAndPlanning(origin, destination, departure_date, return_date):
    # 1. Search for Flights
    print("Searching for flights...")
    orgin_airport_code = get_airport_code(origin)
    destination_airport_code = get_airport_code(destination)
    flights = search_flights(orgin_airport_code, destination_airport_code, departure_date, return_date)
    flights_text = format_travel_data("flights", flights)
    flight_Summary_Detail=flight_Summary(format_flight_data_Content(flights))

    print("Done.")

    # 2. Get AI Flight Recommendation
    print("\nGetting AI flight recommendation...")
    ai_flight_recommendation = get_ai_recommendation("flights", flights_text)
    print("Done.")

    # 3. Search for Hotels
    print("\nSearching for hotels...")
    hotels = search_hotels(destination, departure_date, return_date)
    hotels_text = format_travel_data("hotels", hotels)
    print("Done.")

    # 4. Get AI Hotel Recommendation
    print("\nGetting AI hotel recommendation...")
    ai_hotel_recommendation = get_ai_recommendation("hotels", hotels_text)
    print("Done.")

    # 5. Generate Itinerary
    print("\nGenerating the final itinerary...")
    final_itinerary = generate_itinerary(destination, ai_flight_recommendation, ai_hotel_recommendation, departure_date, return_date,flight_Summary_Detail)
    print("Done.")

    # --- DISPLAY RESULTS ---
    display(Markdown(f"# Your Trip Plan to {destination}"))

    display(Markdown("## ✈️ AI Flight Recommendation"))
    display(Markdown("## ✈️ flight_Summary_Detail"))
    display(Markdown(ai_flight_recommendation))

    display(Markdown("## 🏨 AI Hotel Recommendation"))
    display(Markdown(ai_hotel_recommendation))

    display(Markdown("## 🗺️ Your Complete Itinerary"))
    display(Markdown(final_itinerary))

In [113]:
import json

def OrchesterAgentsAndTask(content):
  output = get_ExtractContent(content)
  final_result_string = processJsonContentForTripPlan(output)
  # Parse the JSON string back into a dictionary
  final_result = json.loads(final_result_string)
  print(final_result['departureDate'])
  # Pass the extracted and processed data to processAndPlanning
  processAndPlanning(
      final_result['originCity'],
      final_result['destinationCity'],
      final_result['departureDate'],
      final_result['returnDate']
  )

In [114]:
text ="Plan a trip from chennai to USA from jan 20 to feb 20 next year"
OrchesterAgentsAndTask(text)

<class 'str'>

Successfully parsed JSON.
2026-01-20
Searching for flights...




> Entering new CrewAgentExecutor chain...


Thought: I now can give a great answer 
Final Answer: 

--- Itinerary 1 (Round trip) ---
**Flight: LH 759 (Lufthansa)**
From: Chennai International Airport (MAA) at 2026-01-20 01:50
To:   Frankfurt Airport (FRA) at 2026-01-20 07:55
Duration: 10h 35m | Class: Economy | Aircraft: Airbus A340
Legroom: 31 in
Extras: Average legroom (31 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 613 kg
 ✦ Overnight flight

**Flight: LH 400 (Lufthansa)**
From: Frankfurt Airport (FRA) at 2026-01-20 11:00
To:   John F. Kennedy International Airport (JFK) at 2026-01-20 13:35
Duration: 8h 35m | Class: Economy | Aircraft: Boeing 747
Legroom: 31 in
Extras: Average legroom (31 in), Wi-Fi for a fee, In-seat power & USB outlets, On-demand video, Carbon emissions estimate: 385 kg
Carbon Emissions: 1000 kg (3% vs typical: 967 kg)
Price: $870
Airline Logo: https://www.gstatic.com/flights/airline_logos/70px/LH.png

--- Itinerary 2 (Round trip) ---
**Flight: EY 341 (Etih

Done.

Searching for hotels...
--- Raw Hotel Data ---
--- End Raw Hotel Data ---
Done.

Getting AI hotel recommendation...


Done.

Generating the final itinerary...
Done.


...

Itinerary 1 (Lufthansa) is the best option. 

Justification: While Itinerary 3 (Emirates) offers the most legroom, Itinerary 1 is the most cost-effective at $870 compared to $939 for Itinerary 2 (Etihad) and $1041 for Itinerary 3 (Emirates). It also has a slightly shorter overall travel time of 19 hours 10 minutes compared to Itinerary 2's 18 hours 50 minutes and Itinerary 3's 19 hours 40 minutes. Although all itineraries have one layover, Itinerary 1's layover in Frankfurt is relatively short, allowing for a quicker connection and less time spent in transit.

## 🏨 AI Hotel Recommendation

Based on the provided data, The Springs Resort is the best recommendation. It boasts an impressive 4.7-star rating from over 11,000 reviews, indicating high customer satisfaction.  Its excellent location (4.4 rating)  and wide array of amenities, including a pool, hot tub, spa, fitness center, and dining options, make it a well-rounded choice.

## 🗺️ Your Complete Itinerary

## 31-Day USA Adventure: An Itinerary Based on Lufthansa Itinerary 1

**All Flight Summary**

**Itinerary 1 (Lufthansa)**

* **Flight LH 759 (Chennai to Frankfurt):** 
    * Depart: Chennai International Airport (MAA) - January 20, 2026, 01:50 AM
    * Arrive: Frankfurt Airport (FRA) - January 20, 2026, 07:55 AM (Local Time)
    * Duration: 10h 35m
    * Aircraft: Airbus A340
    
* **Flight LH 400 (Frankfurt to JFK):**
    * Depart: Frankfurt Airport (FRA) - January 20, 2026, 11:00 AM
    * Arrive: John F. Kennedy International Airport (JFK) - January 20, 2026, 13:35 PM
    * Duration: 8h 35m
    * Aircraft: Boeing 747

**Hotel:** The Springs Resort (Orlando, Florida)

* Check-in: January 20, 2026
* Check-out: February 20, 2026


##  Day 1: Arrival in New York City - The City That Never Sleeps 🌃

*  Arrive at JFK Airport and take the AirTrain and Subway to your hotel in Manhattan 
* Check into your hotel and freshen up.
*  Take a walk through Times Square, soak in the vibrant atmosphere, and capture some iconic photos. 📸
*  Enjoy dinner at a classic New York eatery like Ellen's Stardust Diner for a musical experience 🍽️.

## Day 2: Exploring NYC Highlights ✨

* Start your day with breakfast at Russ & Daughters Cafe in the Lower East Side, known for its delicious bagels 🥯.
* Visit the Statue of Liberty and Ellis Island by ferry. 🚢 Learn about American history and immigration. (Allow 3-4 hours)
*  Walk through Central Park, relax by the Bethesda Fountain, and enjoy the city's green oasis. 🌳
*  In the evening, catch a Broadway show - a quintessential New York experience! 🎭

## Day 3: Museum Mania & Rooftop Views 🖼️🗽

*  Explore the Metropolitan Museum of Art, one of the world's largest and finest art museums. (Allow 3-4 hours)
*  Take a walk across the Brooklyn Bridge for stunning views of the Manhattan skyline. 🌉
*  Enjoy a delicious meal at one of the many rooftop bars in the city, like 230 Fifth Rooftop Bar. 🥂

## Day 4:  Flight to Orlando & Theme Park Fun ✈️🎢

*  Check out of your hotel and take a morning flight from JFK to Orlando International Airport (MCO). 
*  Check into The Springs Resort and head straight to Universal Studios Florida for a day of thrilling rides and movie magic! 🎬
*  Enjoy dinner at one of Universal's themed restaurants.

## Day 5:  More Universal Magic & Thrills 🎢🎡

*  Spend the day exploring Islands of Adventure, home to the Wizarding World of Harry Potter - Hogsmeade. 🧙‍♂️
*  Experience the incredible rollercoasters and attractions at Islands of Adventure.

## Day 6:  Relaxation & Water Park Fun 💦🌴

*  Take a break from the theme parks and relax by the pool at The Springs Resort. 🏊‍♀️
*  In the afternoon, head to Volcano Bay, Universal's water theme park, for a day of water slides, wave pools, and tropical fun. 🌊

## Day 7:  Disney Magic Begins! 🏰🐭

*  Take a day trip to Walt Disney World Resort. 
*  Spend the day at Magic Kingdom, the most iconic of Disney's parks, enjoying classic rides, parades, and fireworks. 🎇

## Day 8:  Exploring Epcot's Wonders 🌎🚀

*  Visit Epcot, known for its futuristic vision and world cultures. 
*  Explore Future World and enjoy the innovative attractions and pavilions.
*  Take a journey around the World Showcase, sampling international cuisine and experiencing different cultures. 

## Day 9:  Animal Kingdom Adventures 🦓🐘

*  Spend the day at Disney's Animal Kingdom, home to amazing wildlife and exciting attractions. 
*  Experience the Kilimanjaro Safaris, ride the Avatar Flight of Passage, and explore Pandora - The World of Avatar. 🌿

## Day 10:  Hollywood Studios & Star Wars 🎬🌌

*  Visit Disney's Hollywood Studios, where movie magic comes to life. 
*  Experience the thrilling Twilight Zone Tower of Terror, the immersive Star Wars: Galaxy's Edge, and the exciting Slinky Dog Dash.

## Days 11-14:  Exploration of Florida's Coasts & Keys 🌴☀️🌊

*  Take a day trip to Clearwater Beach, known for its white sand beaches and clear waters. 🏖️
*  Explore St. Augustine, the oldest city in the United States, with its historic architecture and charming atmosphere. 🏛️
*  Embark on a scenic drive down the Overseas Highway to Key West, the southernmost point in the continental US. 🚗🌴
*  Enjoy snorkeling, diving, or simply relaxing on the beautiful beaches of the Florida Keys. 🐠

## Days 15-18:  Back to Orlando & Other Attractions

* Visit Kennedy Space Center Visitor Complex and learn about the history of space exploration. 🚀
*  Explore ICON Park, home to The Wheel, Madame Tussauds Orlando, and SEA LIFE Aquarium. 🎡
*  Enjoy a day of shopping at Orlando International Premium Outlets. 🛍️

## Days 19-21:  Relaxation & Departure Preparation

*  Spend a few days relaxing by the pool at The Springs Resort 
*  Catch up on any souvenirs you might want to buy. 
*  Start packing and preparing for your departure.

## Day 22:  Departure ✈️

*  Enjoy a final breakfast in Orlando. 
*  Check out of The Springs Resort and head to Orlando International Airport (MCO) for your flight back to Chennai.

##